### Table of contents:

* [Investments](#investments)
* [Investors](#investors)
* [Funding rounds](#fun_rounds)
* [Merge investments and funding rounds](#merge)

In [1]:
import math
import pandas as pd
import arrow
import ipynb 
import os.path
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from networkx.algorithms import bipartite
from importlib import reload

load_dotenv(verbose=True)


import pandas as pd
pd.options.mode.chained_assignment = None  

In [2]:
# import functions from py file 

import functions.fun
reload(functions.fun)
from functions.fun import CB_data_cleaning, df_from_api_CB, extract_nodes, extract_data_from_column
from functions.fun import nx_dip_graph_from_pandas, plot_bipartite_graph, filter_dict
from functions.fun import extract_classes_company_tech, degree_bip, insert_data_classes

In [3]:
# import classes 
import classes

## Investors <a class="anchor" id="investors"></a>

In [4]:
df_investors = pd.read_csv("data/data_cb/investors.csv")

In [5]:
df_investors.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,roles,domain,...,investment_count,total_funding_usd,total_funding,total_funding_currency_code,founded_on,closed_on,facebook_url,linkedin_url,twitter_url,logo_url
0,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,6766.0,2007-05-26 02:30:28,2018-10-27 00:29:49,"investor,company",zoho.com,...,5.0,NaN,NaN,NaN,1996-09-15,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...
1,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,88317.0,2007-05-26 03:21:34,2019-06-19 12:17:48,investor,omidyar.com,...,318.0,NaN,NaN,NaN,2004-01-01,NaN,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://res.cloudinary.com/crunchbase-producti...
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,25.0,2007-05-26 04:22:15,2021-05-17 14:06:02,"investor,company",facebook.com,...,37.0,1.612282e+10,1.612282e+10,USD,2004-02-04,NaN,https://www.facebook.com/facebook,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://res.cloudinary.com/crunchbase-producti...
3,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,person,mark-zuckerberg,https://www.crunchbase.com/person/mark-zuckerberg,2220.0,2007-05-26 04:51:46,2018-12-13 14:05:34,investor,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/zuck,NaN,https://twitter.com/finkd,https://res.cloudinary.com/crunchbase-producti...
4,084aaa07-0795-1fe8-9c46-98bbeb02cd64,Dustin Moskovitz,person,dustin-moskovitz,https://www.crunchbase.com/person/dustin-mosko...,93903.0,2007-05-26 05:15:05,2019-06-07 20:29:51,investor,NaN,...,16.0,NaN,NaN,NaN,NaN,NaN,http://www.facebook.com/moskov,https://www.linkedin.com/in/dmoskov,http://twitter.com/moskov,https://res.cloudinary.com/crunchbase-producti...


In [6]:
df_investors.loc[0]

uuid                                        bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7
name                                                                        Zoho
type                                                                organization
permalink                                                                   zoho
cb_url                              https://www.crunchbase.com/organization/zoho
rank                                                                      6766.0
created_at                                                   2007-05-26 02:30:28
updated_at                                                   2018-10-27 00:29:49
roles                                                           investor,company
domain                                                                  zoho.com
country_code                                                                 USA
state_code                                                                    CA
region                      

## Investments <a class="anchor" id="investments"></a>

In [57]:
df_investments = pd.read_csv("data/data_cb/investments.csv")

In [58]:
df_investments.columns

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'funding_round_uuid', 'funding_round_name',
       'investor_uuid', 'investor_name', 'investor_type', 'is_lead_investor'],
      dtype='object')

**Data Cleaning**

In [69]:
to_drop = [
    'uuid',
    'permalink',
    'funding_round_name',
    'cb_url',
    'created_at',
    'updated_at',   
    'rank',
    'created_at',
    'updated_at'
]

# to_rename = { 'category_groups_list': 'category_groups' }
to_rename = {  }

drop_if_nan = [

]

to_check_double = {}

sort_by = ""

In [70]:
df_investments = CB_data_cleaning(df_investments, to_drop, to_rename, to_check_double, drop_if_nan, sort_by)

In [71]:
df_investments.head()

,name,type,funding_round_uuid,investor_uuid,investor_name,investor_type,is_lead_investor
0,Accel investment in Series A - Facebook,investment,d950d7a5-79ff-fb93-ca87-13386b0e2feb,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,organization,True
1,Greylock investment in Series B - Facebook,investment,6fae3958-a001-27c0-fb7e-666266aedd78,e2006571-6b7a-e477-002a-f7014f48a7e3,Greylock,organization,True
2,Meritech Capital Partners investment in Series...,investment,6fae3958-a001-27c0-fb7e-666266aedd78,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital Partners,organization,True
3,Trinity Ventures investment in Series B - Phot...,investment,bcd5a63d-ed99-6963-0dd2-e36f6582f846,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures,organization,NaN
4,Founders Fund investment in Series A - Geni,investment,60e6afd9-1215-465a-dd17-0ed600d4e29b,fb2f8884-ec07-895a-48d7-d9a9d4d7175c,Founders Fund,organization,True


In [72]:
df_investments.loc[0]

name                  Accel investment in Series A - Facebook
type                                               investment
funding_round_uuid       d950d7a5-79ff-fb93-ca87-13386b0e2feb
investor_uuid            b08efc27-da40-505a-6f9d-c9e14247bf36
investor_name                                           Accel
investor_type                                    organization
is_lead_investor                                         True
Name: 0, dtype: object

## Funding rounds <a class="anchor" id="fun_rounds"></a>

In [73]:
df_funding = pd.read_csv("data/data_cb/funding_rounds.csv")

In [74]:
to_drop = [
    'type',
    'permalink',
    'cb_url',   
    'rank',
    'funding_round_name',
    'created_at',
    'updated_at',
    'investor_type',
    'raised_amount',
    'is_lead_investor',
    'post_money_valuation_usd',                                      
    'post_money_valuation',                                    
    'post_money_valuation_currency_code',
]

# to_rename = { 'category_groups_list': 'category_groups' }
to_rename = { 'category_list': 'category_groups', 'uuid':'funding_round_uuid' }

drop_if_nan = [
    'name'
]

to_check_double = {}

sort_by = ""

In [75]:
df_funding = CB_data_cleaning(df_funding, to_drop, to_rename, to_check_double, drop_if_nan, sort_by)

In [79]:
df_funding.head()


,funding_round_uuid,name,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,USA,CA,California,Menlo Park,series_b,2006-04-01,27500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,USA,CO,Colorado,Denver,series_b,2006-05-01,10500000.0,USD,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,USA,CA,California,West Hollywood,series_a,2007-01-17,NaN,NaN,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c


In [80]:
df_funding.loc[0]

funding_round_uuid             8a945939-18e0-cc9d-27b9-bf33817b2818
name                                         Angel Round - Facebook
country_code                                                    USA
state_code                                                       CA
region                                                   California
city                                                     Menlo Park
investment_type                                               angel
announced_on                                             2004-09-01
raised_amount_usd                                          500000.0
raised_amount_currency_code                                     USD
investor_count                                                  4.0
org_uuid                       df662812-7f97-0b43-9d3e-12f64f504fbb
org_name                                                   Facebook
lead_investor_uuids            3f47be49-2e32-8118-01a0-31685a4d0fd7
Name: 0, dtype: object

## Merge investments and funding rounds <a class="anchor" id="merge"></a>

The two datasets invetsments and funding rounds must be matched in order to have infomrations about the investments and the amount of the invetsments (contained only in the funding rounds dataset)

In [105]:
df_invest_funding = pd.merge(df_funding, df_investments, on='funding_round_uuid')

In [106]:
to_drop = [
    'funding_round_uuid',
    'name_x',
    'org_uuid',   
    'lead_investor_uuids',
    'name_y',
    'investor_uuid',
]

# to_rename = { 'category_groups_list': 'category_groups' }
to_rename = {  }
drop_if_nan = []
to_check_double = {}
sort_by = ""

In [107]:
df_invest_funding = CB_data_cleaning(df_invest_funding, to_drop, to_rename, to_check_double, drop_if_nan, sort_by)

In [108]:
df_invest_funding.head()

,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount_currency_code,investor_count,org_name,type,investor_name,investor_type,is_lead_investor
0,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,USD,4.0,Facebook,investment,Peter Thiel,person,True
1,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,USD,4.0,Facebook,investment,Reid Hoffman,person,False
2,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,USD,4.0,Facebook,investment,Mark Pincus,person,False
3,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,USD,4.0,Facebook,investment,Western Technology Investment,organization,False
4,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,USD,4.0,Facebook,investment,Accel,organization,True


## Analysis

In [109]:
df_essential = df_invest_funding[["org_name", "investor_name", "raised_amount_usd"]]
df_essential.head()

,org_name,investor_name,raised_amount_usd
0,Facebook,Peter Thiel,500000.0
1,Facebook,Reid Hoffman,500000.0
2,Facebook,Mark Pincus,500000.0
3,Facebook,Western Technology Investment,500000.0
4,Facebook,Accel,12700000.0


In [110]:
# delete value with NaN investments

df_essential.dropna(subset = ["raised_amount_usd"], inplace=True)

df_essential.head()

,org_name,investor_name,raised_amount_usd
0,Facebook,Peter Thiel,500000.0
1,Facebook,Reid Hoffman,500000.0
2,Facebook,Mark Pincus,500000.0
3,Facebook,Western Technology Investment,500000.0
4,Facebook,Accel,12700000.0


In [111]:
len(df_essential)

494133

In [112]:
# total amount of investments for each company

df_num_inves_per_company = df_essential.groupby("org_name").sum()
df_num_inves_per_company = df_num_inves_per_company.sort_values("raised_amount_usd", ascending=False)
df_num_inves_per_company.head()

,raised_amount_usd
org_name,
Broadcom Limited,8.000000e+11
Ant Group,1.580000e+11
Abu Dhabi National Oil Company,1.400000e+11
Uber,1.345116e+11
Didi,1.004617e+11


In [113]:
# total amount of investments for each investors

df_num_inves_per_company = df_essential.groupby("investor_name").sum()
df_num_inves_per_company = df_num_inves_per_company.sort_values("raised_amount_usd", ascending=False)
df_num_inves_per_company.head()

,raised_amount_usd
investor_name,
Silver Lake,1.627348e+11
Kohlberg Kravis Roberts,1.521948e+11
JP Morgan Chase,1.445013e+11
Tencent,1.381565e+11
Morgan Stanley,1.315562e+11


In [114]:
# number of investments for each company

df_num_inves_per_company = df_essential[["org_name", "investor_name"]].groupby("org_name").count()
df_num_inves_per_company = df_num_inves_per_company.sort_values("investor_name", ascending=False)
df_num_inves_per_company = df_num_inves_per_company.rename(columns={"investor_name":"number_inv_rounds"})
df_num_inves_per_company.head()

,number_inv_rounds
org_name,
Gusto,132
Uber,120
Dapper Labs,103
Lyft,100
Bolt,100


In [115]:
# number of investments for each investors

df_num_inves_per_company = df_essential[["org_name", "investor_name"]].groupby("investor_name").count()
df_num_inves_per_company = df_num_inves_per_company.sort_values("org_name", ascending=False)
df_num_inves_per_company = df_num_inves_per_company.rename(columns={"org_name":"number_investments"})
df_num_inves_per_company.head()

,number_investments
investor_name,
EASME - EU Executive Agency for SMEs,3672
Y Combinator,2563
Techstars,2027
New Enterprise Associates,1737
500 Startups,1609


### Graph

In [116]:
df_graph = df_essential[["org_name", "investor_name"]]
df_graph = df_graph.set_index("org_name")
df_graph.head()

,investor_name
org_name,
Facebook,Peter Thiel
Facebook,Reid Hoffman
Facebook,Mark Pincus
Facebook,Western Technology Investment
Facebook,Accel


In [117]:
# import functions from py file 

import functions.fun
reload(functions.fun)
from functions.fun import nx_dip_graph_from_pandas, plot_bipartite_graph, filter_dict, df_from_api_CB, extract_nodes

In [118]:
# set limit
df_graph = df_graph[:100]

In [119]:
# create the bipartite network

B = nx_dip_graph_from_pandas(df_graph)

{'Facebook': array([0.35791877, 0.50398409]),
 'Peter Thiel': array([0.35328932, 0.60235431]),
 'Reid Hoffman': array([0.2971415 , 0.48417183]),
 'Mark Pincus': array([0.29885285, 0.27390786]),
 'Western Technology Investment': array([0.2559721 , 0.39854774]),
 'Accel': array([0.37056376, 0.30965559]),
 'Cambrian Ventures': array([0.55475398, 0.57282506]),
 'Greylock': array([0.48402928, 0.67741184]),
 'Meritech Capital Partners': array([0.5208094 , 0.46385489]),
 'Founders Fund': array([0.53173374, 0.63292749]),
 'SV Angel': array([0.00709573, 0.56429771]),
 'Photobucket': array([0.06895921, 0.9946259 ]),
 'Trinity Ventures': array([0.08441967, 0.88346129]),
 'Chris Sacca': array([-0.01947628,  0.96389291]),
 'Geni': array([-0.91825014, -0.21990418]),
 'CRV': array([-0.84402577, -0.25231319]),
 'StumbleUpon': array([0.149586  , 0.15961416]),
 'First Round Capital': array([0.00031478, 0.0523306 ]),
 'Ram Shriram': array([0.34610049, 0.14743721]),
 'Mitchell Kapor': array([0.36353479, 0

In [120]:
plot_bipartite_graph(B, small_degree=True)

{'Facebook': array([-0.75666987, -0.24517314]),
 'Peter Thiel': array([-0.60785678, -0.16324502]),
 'Reid Hoffman': array([-0.6201887, -0.3767687]),
 'Mark Pincus': array([-0.76219962, -0.40563908]),
 'Western Technology Investment': array([-0.66643102, -0.28853221]),
 'Accel': array([-0.77222281, -0.11213036]),
 'Cambrian Ventures': array([-0.84414005, -0.00285214]),
 'Greylock': array([-0.88341449, -0.36806209]),
 'Meritech Capital Partners': array([-0.87933916, -0.19765695]),
 'Founders Fund': array([-0.88491952, -0.28908473]),
 'SV Angel': array([-0.66433529, -0.39117342]),
 'Photobucket': array([0.02490369, 0.78061436]),
 'Trinity Ventures': array([-0.0440054 ,  0.94330119]),
 'Chris Sacca': array([0.04390537, 0.54927152]),
 'StumbleUpon': array([ 0.59066924, -0.07107628]),
 'First Round Capital': array([ 0.45144835, -0.34995743]),
 'Ram Shriram': array([0.64432184, 0.11260245]),
 'Mitchell Kapor': array([ 0.70756487, -0.00758216]),
 'Ronald Conway': array([0.61022652, 0.14404777]

In [121]:
plot_bipartite_graph(B, small_degree=False)

### Add weights to the graph

In [122]:
df_essential.head()

,org_name,investor_name,raised_amount_usd
0,Facebook,Peter Thiel,500000.0
1,Facebook,Reid Hoffman,500000.0
2,Facebook,Mark Pincus,500000.0
3,Facebook,Western Technology Investment,500000.0
4,Facebook,Accel,12700000.0


In this way, df_essential contains the sum of all the investments from one investor to a specific company.

For later: check if data of investments may be useful

In [123]:
df_essential = df_essential.groupby(["org_name", "investor_name"]).sum("raised_amount_usd")
df_essential.head()

raised_amount_usd
org_name      investor_name                        
!hey software Goldman Sachs              10000000.0
              Telcom Ventures            10000000.0
#paid         Aitken Investments          5000000.0
              Angel One Network            500000.0
              Freycinet Ventures          5000000.0

In [103]:
# add weights

